## 使用NaiveBayes分类器检测垃圾短信 ##

数据集: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

通常而言, 垃圾短信包含一些特殊吸引眼球的单词, 例如‘free’, ‘win’, ’prize‘ 等等, 这些恰好是我们可以用于训练机器学习模型的好特征. 此任务为一个二元分类的任务, 短信的类别为‘Spam’(垃圾短信)或者‘Not Spam’(正常短信)两类. 

# 大纲

此项目分为如下步骤:
- 1.1: 分析短信数据
- 1.2: 数据预处理
- 2.1: 词袋(Bag of Words)模型
- 2.2: 从头开始自己实现词袋(Bag of Words)模型
- 2.3: 使用现成的scikit-learn中的词袋(Bag of Words)模型
- 3.1: 准备训练和测试数据集
- 3.2: 使用词袋模型处理短信数据集
- 4.1: 理解Bayes理论
- 4.2: 从头开始自己实现NaiveBayes分类器
- 5:   使用现成的scikit-learn中的NaiveBayes分类器
- 6:   结论



### 1.1: 分析短信数据 ### 


数据来源于https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection, 可以通过如下链接下载:(https://archive.ics.uci.edu/ml/machine-learning-databases/00228/).


 **部分数据:** 

<img src="images/dqnb.png" height="1242" width="1242">

数据包含两列, 第一列是每一条短信的标签‘ham’或者‘spam’. 第二列为短信文本.


In [3]:
import pandas as pd
df = pd.read_csv('smsspamcollection/SMSSpamCollection', sep='\t', names=['label', 'message'])

df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### 1.2: 数据预处理 ###

将类别标签从字符串‘ham', ’spam'转换为数字 0 和 1.


In [4]:
df['label'] = df.label.map({'ham':0, 'spam':1})
print(df.shape)
df.head()

(5572, 2)


,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### 2.1: 词袋(Bag of Words)模型 ### 

词袋(Bag of Words)模型: 统计一段文字中出现的每一个单词的次数, 不考虑单词出现的顺序/位置.

通过词袋模型, 我们可以将一个包含多个文本的数据集转换为一个矩阵, 矩阵的每一行对应一个文本, 每一列是某一个单词出现在当前这一文本中的次数.例如, 我们有如下四个文本:

`['Hello, how are you!',
'Win money, win from home.',
'Call me now',
'Hello, Call you tomorrow?']`

通过词袋模型, 我们将它转换为如下矩阵:

<img src="images/countvectorizer.png" height="542" width="542">

我们可以使用 sklearns 中的方法来实现这个操作 
[count vectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) :

* 首先将一段文字分为多个独立的单词, 使用一个整数代表每一个单词.
* 计算每一个单词在这段文字中出现的次数.

**注意:** 

* CountVectorizer 会把所有的单词自动转换为小写的形式, 所以 'He' 和 'he' 会被当作同一个单词. 

* 会去掉所有的标点符号, 例如'hello!'和'hello'会被当作一个单词. 

* 这个方法允许传入参数 `stop_words` , 用于指代需要去掉传入的语言中太常用的词. 例如英文中的'am', 'an', 'and', 'the' 等等. 如果将 `stop_words` 的值为 `english`, CountVectorizer 方法会自动使用scikit-learn自带的英文 `stop_words` 去掉英文中的常用词. 通常这些 `stop_words` 对于我们判断一个文本是否是垃圾短信没有帮助.


### 2.2: 从头开始自己实现词袋(Bag of Words)模型 ###

在使用 scikit-learn 内建的词袋(Bag of Words)库之前, 我们可以试着自己动手实现词袋模型. 

**1: 将所有的单词专为小写.**

假设我们的文档如下:

```python
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']
```



In [5]:
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

lower_case_documents = []
for i in documents:
    lower_case_documents.append(i.lower())
print(lower_case_documents)

['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


**2: 去掉标点符号**

 

In [6]:
sans_punctuation_documents = []
import string

for i in lower_case_documents:
    sans_punctuation_documents.append(i.translate(str.maketrans('', '', string.punctuation)))
    
print(sans_punctuation_documents)

['hello how are you', 'win money win from home', 'call me now', 'hello call hello you tomorrow']


**3: 将句子分割为多个单词**



In [7]:
preprocessed_documents = []
for i in sans_punctuation_documents:
    preprocessed_documents.append(i.split())
print(preprocessed_documents)

[['hello', 'how', 'are', 'you'], ['win', 'money', 'win', 'from', 'home'], ['call', 'me', 'now'], ['hello', 'call', 'hello', 'you', 'tomorrow']]


**4: 计算单词在每一个句子中出现的次数**

In [8]:
frequency_list = []
import pprint
from collections import Counter

for i in preprocessed_documents:
    frequency_list.append(Counter(i))
    
pprint.pprint(frequency_list)

[Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1}),
 Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}),
 Counter({'call': 1, 'me': 1, 'now': 1}),
 Counter({'hello': 2, 'call': 1, 'you': 1, 'tomorrow': 1})]


以上就是scikit-learn中 `sklearn.feature_extraction.text.CountVectorizer` 方法的原理. 接下来我们是scikit-learn来完成同样的任务.


### 2.3: 使用scikit-learn的词袋模型 ###



In [42]:
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']

from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

**CountVectorizer()中的数据预处理**

注意, CountVectorizer() 默认使用如下的数据预处理:

* `lowercase = True`
    
    将所有单词专为小写.


* `token_pattern = (?u)\\b\\w\\w+\\b`
    
    这个参数的意思是使用如上的正则表达式去掉所以的标点符号, 把标点符号的位置当作单词的分界点. 去掉长度小于2的单词.


* `stop_words`

        默认为 `None` 如果设为 `english` 则去掉所有的英文常用词, 考虑到我们的数据是短信, 每一个单词都非常重要, 我们就不去常用词了. 

In [12]:
'''
察看 'CountVectorizer() 的默认参数'
'''
print(count_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [15]:
# 得到出现在数据集中的所有单词
count_vector.fit(documents)
count_vector.get_feature_names()

['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

In [16]:
# 将文本专为矩阵, 每一行对应一条短信文本, 每一列对应一个单词在短信中出现的次数
doc_array = count_vector.transform(documents).toarray()
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

我们可以把这个矩阵转为 panda.dataframe, 并且每一列标上单词, 以便察看.

In [17]:
frequency_matrix = pd.DataFrame(doc_array, columns=count_vector.get_feature_names())
frequency_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


注意我们没有去掉常用词, 例如 'are', 'is', 'the', 'an', 他们可能会影响预测. 为了解决这个问题, 有两种方案:

1. 设置 `stop_words` 的值为 `english`, 去除常用词. 

2. 使用 [tfidf](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer) 而不单单考虑单词出现的频率. 

### 3.1: 准备训练和测试数据集 ###



In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['message'], 
                                                    df['label'], 
                                                    random_state=1)

print('总的短信数量: {}'.format(df.shape[0]))
print('训练集的短信数量: {}'.format(X_train.shape[0]))
print('测试集的短信数量: {}'.format(X_test.shape[0]))

总的短信数量: 5572
训练集的短信数量: 4179
测试集的短信数量: 1393


### 3.2: 使用词袋模型处理短信数据集 ###

注意: 代码分为两步, 第一步使用训练数据集获取所有的单词集合, 同时将训练数据集转为矩阵. 第二步直接使用第一步获取的单词集合, 将测试数据集转为矩阵.


In [26]:
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)
print(testing_data[:2])

  (0, 1538)	1
  (0, 5189)	1
  (0, 6542)	1
  (0, 7405)	1
  (1, 1016)	1
  (1, 3050)	1
  (1, 4163)	1
  (1, 4238)	1
  (1, 4370)	1
  (1, 5200)	1
  (1, 6656)	1
  (1, 7407)	1
  (1, 7420)	1


### 4.1: 理解Bayes理论 ###

我们使用一个简单的例子来理解Bayes理论.

假设:

`P(D)` 为任何一个人罹患糖尿病的概率, 假设其值为 `0.01`.

`P(Pos)` 为任何一个人做糖尿病检测时, 结果为阳性的的概率.

`P(Pos|D)` 为任何一个已经罹患糖尿病人做糖尿病检测时, 结果为阳性的的概率, 其值为 `0.9`.

`P(Pos|~D)` 为任何一个没有罹患糖尿病人做糖尿病检测时, 结果为阳性的的概率, 其值为 `0.1`.

求: 一个人检测结果为阳性的情况下, 这个人真正罹患糖尿病的概率P(D|Pos).

贝叶斯公式如下:

<img src="images/bayes_formula.png" height="242" width="242">

根据Bayes公式:

`P(D|Pos) = P(Pos|D) * P(D) / P(Pos)`

根据全概率公式:

`P(Pos) = [P(D) * P(Pos|D)] + [P(~D) * P(Pos|~D)]`

In [44]:
# P(D)
p_diabetes = 0.01

# P(~D)
p_no_diabetes = 0.99

# P(Pos|D)
p_pos_diabetes = 0.9

# P(Pos|~D)
p_pos_no_diabetes = 0.1

# P(Pos)
p_pos = (p_diabetes * p_pos_diabetes) + (p_no_diabetes * p_pos_no_diabetes)
print('任何一个人(无论是否罹患糖尿病)检测结果为阳性的概率 P(Pos): {}'.format(p_pos))

# P(D|Pos)
p_diabetes_pos = (p_diabetes * p_pos_diabetes) / p_pos
print('一个人检测结果为阳性的情况下, 这个人真正罹患糖尿病的概率:',format(p_diabetes_pos)) 

任何一个人(无论是否罹患糖尿病)检测结果为阳性的概率 P(Pos): 0.10800000000000001
一个人检测结果为阳性的情况下, 这个人真正罹患糖尿病的概率: 0.08333333333333333


意味着即使被测出来为阳性, 真正罹患糖尿病的概率为 8.3%, 不要太担心, 需要再次复查确诊. 当然前提是我们假设只有 1% 的人真正有糖尿病.

**'Naive Bayes'(朴素贝叶斯)中的'Naive'(朴素)的意义 ?** 

'Naive'指预测的特征之间是独立的. 例如糖尿病的例子, 假设我们的除了检测结果是否为阳性, 还有另外一个特征是这个人的体重. 'Naive Bayes'在计算的时候假设‘检测结果是否为阳性’和‘体重’是独立的, 相互不影响. 

### 4.2: 从头开始自己实现NaiveBayes分类器 ###



假设:

* 假设正常短信出现单词'good'的概率: 0.7 -----> `P(g|h)`
* 假设正常短信出现单词'free'的概率: 0.1 -----> `P(f|h)`
* 假设正常短信出现单词'win'的概率: 0.2  -----> `P(w|h)`


* 假设垃圾短信出现单词'good'的概率: 0.3 -----> `P(g|s)`
* 假设垃圾短信出现单词'free'的概率: 0.9 -----> `P(f|s)`
* 假设垃圾短信出现单词'win'的概率: 0.8  -----> `P(w|s)`


假设正常短信和垃圾短信出现的概率分别为 `P(h) = 0.8`, `P(s) = 0.2` 

Naive Bayes 公式为:

<img src="images/naivebayes.png" height="342" width="342">

求:

一条短信出现了’free‘和’win', 这条短信为垃圾短信的概率?

我们需要如下计算:

* `P(s|f,w)`: 出现‘free’和‘win'的时候, 短信为垃圾短信的概率. 

* `P(s|f,w)` = `(P(s) * P(f|s) * P(w|s)) / P(f,w)`: 其中 `P(f,w)` 为一条短信中同时包含‘free'和’win'的概率.
    
* `P(f,w) = P(h) * P(f, w | h) + P(s) * P(f, w | s)`

使用‘Naive’的方式, 认为 f, w 这两个特征是独立的
* `P(f,w) = P(h) * P(f | h) * P(w | h) + P(s) * P(f | s) * P(w | s)`

In [45]:
# P(s)
p_s = 0.2

# P(f/s)
p_f_s = 0.9

# P(w/s)
p_w_s = 0.8

# P(h)
p_h = 0.8

# P(f/h)
p_f_h = 0.1

# P(w|h)
p_w_h = 0.2

#`P(f,w) = P(h) * P(f, w | h) + P(s) * P(f, w | s)`
p_fw = p_h * p_f_h * p_w_h + p_s * p_f_s * p_w_s
print('短信同时出现free和win的概率:{0}'.format(p_fw))

#`P(s|f,w)` = `(P(s) * P(f|s) * P(w|s)) / P(f,w)`
p_s_fw = p_s * p_f_s * p_w_s / p_fw
print('短信同时出现free和win时为垃圾短信的概率:{0}'.format(p_s_fw))

短信同时出现free和win的概率:0.16000000000000003
短信同时出现free和win时为垃圾短信的概率:0.8999999999999999


### 5: 使用现成的scikit-learn中的NaiveBayes分类器 ###

`sklearn.naive_bayes` 包含现成的方式实现NaiveBayes分类器, 其中分为两种情况, 一种是MultinomialNB, 用于特征为离散值的情况, 另一种为GuassianNB, 用于特征值为连续值的情况. 

In [39]:
# 训练
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
# 预测
predictions = naive_bayes.predict(testing_data)

#### 评价模型性能 ####


**Accuracy**: 分类器预测的结果为正确类别的数量占测试数据集中所有短信数量的百分比.

**Precision**: 分类器预测为垃圾短信的短信中, 其真实情况为垃圾短信的百分比.
为 true positives (真实垃圾短信, 被预测为垃圾短信) 除以所有被预测为垃圾短信的数量(无论是否为垃圾短信, 预测为垃圾短信的数量)

`[True Positives/(True Positives + False Positives)]`

**Recall(sensitivity)**: 测试数据集中所有真实垃圾短信被预测为垃圾短信的百分比.  
为 true positives (真实垃圾短信, 被预测为垃圾短信) 除以所有垃圾短信的数量

`[True Positives/(True Positives + False Negatives)]`

**F1-Score**: 综合考虑 Precision 和 Recall 的情况, 取值范围为 0 到 1 之间, 越大越好.
`2 * Precision * Recall / (Precision + Recall) `

在类别不平衡的情况下, 例如100条短信只有 1 条为垃圾短信的情况下, **Accuracy**并不是好的评价指标, 因为只要盲目的预测任意短信为正常短信, **Accuracy**就为99%. 这时使用F1-Score是更好的选择.


In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9885139985642498
Precision score:  0.9720670391061452
Recall score:  0.9405405405405406
F1 score:  0.9560439560439562


### 6. 结论 ###

NaiveBayes分类器的优势
* 1. 可以处理特征数量巨大的情况, 自然语言处理常常是这样的情况. 
* 2. 算法简单, 容易理解
* 3. 不容易过拟合
* 4. 训练时间短

NaiveBayes分类器的不足:
* 1. 假设特征之间是独立的, 可能在有的情况下这种假设会造成问题
* 2. 需要足够多的数据才能获得特征比较准确的概率分布
